<a href="https://colab.research.google.com/github/Francrodi/AprendAut_T2/blob/master/Cliente_Lab2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re


filename = "chat.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

messages_list = []

for line in data:
  parts = line.split(":")
  message = parts[-1].strip()
  messages_list.append(message)

def preprocess_chat(chat: list) -> list:
  chat = [message.lower() for message in chat]
  chat = [re.sub(r"[^a-z\s]", "", message) for message in chat]
  chat = [re.sub(r"HASHTAG|URL|MENTION", "", message) for message in chat]
  chat = [message.strip() for message in chat]
  chat = [message for message in chat if message != ""]
  chat = [re.sub(r"\s+", " ", message) for message in chat]
  chat = list(dict.fromkeys(chat))
  chat.sort()
  return chat

messages_list = preprocess_chat(messages_list)

messages_list

In [ ]:
# Crear una lista con todas las palabras de los mensajes
words_list = []

for message in messages_list:
  words = message.split(" ")
  for word in words:
    words_list.append(word)

# Crear un diccionario con las palabras y la cantidad de veces que aparecen
words_dict = {}

for word in words_list:
  if word in words_dict:
    words_dict[word] += 1
  else:
    words_dict[word] = 1

words_dict

In [ ]:
# Lista de palabras unicas (Vocabulario)
vocabulary = list(dict.fromkeys(words_list))

# Construimos diccionario de probabilidades de las palabras P(H)
total_words = sum(words_dict.values())
ph_dict = {word: frequency / total_words for word, frequency in words_dict.items()}

# Construimos diccionario de probabilidades P(D|H) en base al hiperparametro N (Cantidad de palabras a considerar)
# TODO: Revisar si se esta haciendo correctamente
def build_pdh_dict(chat: list, N=2: int):
  pdh_dict = {}
  for message in chat:
    words = message.split(" ")
    for i in range(len(words) - N + 1):
      key = " ".join(words[i:i+N-1])
      value = words[i+N-1]
      if key in pdh_dict:
        if value in pdh_dict[key]:
          pdh_dict[key][value] += 1
        else:
          pdh_dict[key][value] = 1
      else:
        pdh_dict[key] = {value: 1}

  for key, value in pdh_dict.items():
    total = sum(value.values())
    pdh_dict[key] = {k: v / total for k, v in value.items()}

  return pdh_dict

pdh_dict = build_pdh_dict(messages_list, N=2)

pdh_dict

In [ ]:
# TODO: Revisar, seguramente este mal
def recomendacion_bayesiana(frase):
  if len(frase) >= 2:
    keys = frase[-2:]
  else:
    keys = frase

  keys = [k.lower() for k in keys]

  # Bucar la palabra con mayor probabilidad, es decir, que maximice P(H|D) = P(D|H) * P(H) (Hipoteses MAP)
  max_prob = 0
  palabra_sugerida = ""
  for word in vocabulary:
    prob = 0
    for key in keys:
      prob *= pdh_dict[word].get(key, 0.001)
    prob *= ph_dict[word]
    if prob >= max_prob:
      max_prob = prob
      palabra_sugerida = word

  return palabra_sugerida

def preprocesar_chat(chat):
  # algo aca
  pass

def entrenar():
  # Tener en cuenta el N
  # Armar una estructura tipo diccionario de profundidad N
  pass



##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))



# hola chau buen dia PRED

# p(buen|PRED)*p(dia|PRED)
# p[PRED]["dia"]*p[PRED]["buen"]*p(PRED)



Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
>> asdf
asdf Martes
>> 
asdf Martes Domingo
>> .
----- Comenzando frase nueva -----
>> hola
hola Viernes


KeyboardInterrupt: ignored